# 사용자 기반 CF와 아이템 기반 CF 

- CF 추천 알고리즘은 사용자를 기준으로 비슷한 취향의 이웃을 선정하는 방식을 사용하였다. 이런 방식을 사용자 기반 CF(User-Based CF: UBCF)라고 부르며, 반대로 아이템을 기준으로 하는 아이템 기반 CF(Item-Based CF: IBCF)도 가능하다 
- 이 둘의 차이는 유사도를 계산하는 기준이 사용자인가 아이템인가 하는 것이다 
- UBCF는 취향이 비슷한 이웃 사용자(neighbor)를 알아내고, 이 그룹에 속한 사용자들이 공통적으로 좋게 평가한 아이템을 추천하는 방식
- IBCF는 반대로 사용자들의 평가 패턴을 바탕으로 아이템 간의 유사도를 계산해서 아용자의 특정 아이템에 대한 예측 평점을 계산하는 방식이다. 좀더 구체적으로는 에측 대상 사용자가 평가한 아이템의 평점과, 다른 각 아이템과의 유사도를 가중해서 평균한 값을 그 아이템에 대한 에측값으로 사용 한다

- 이 두가지 방식은 장단점이 있다. 우선 UBCF는 각 사용자별로 맞춤형 추천을 하기 때문에 데이터가 풍부한 경우 정확한 추천이 가능하다. 반대로 IBCF는 정확도는 떨어지지만 사용자별로 따로따로 게산을 하지 않기 때문에 계산이 빠르다는 장점이 있다. 그리고 UBCF는 정확할 때는 매우 정확하지만 터무니 없는 추천을 하는 경우도 상당히 있는 데 비해 IBCF는 그럴 위험이 적다. 또한 UBCF는 데이터가 조금 바뀔 때마다 업데이트를 해야 하지만 IBCF는 데이터가 조금 바뀌어도 추천 결과에는 영향이 크지 않기 때문에 업데이트를 자주 하지 않아도 된다

- 종합하자면 데이터 크기가 적고 각 사용자에 대한 충분한 정보(구매나 평가)가 있는 경우에는 UBCF가 알맞고
- 데이터가 크거나 각 사용자에 대한 충분한 정보가 없는 경우에는 IBCF가 알맞다고 할 수 있다 


- 데이터가 충분하다면 UBCF가 IBCF보다 다수 정확하다고 알려져 있다. 대신에 IBCF가 계산이 빠르기 때문에 아마존과 같은 대규모 데이터를 다뤄야 하는 상업용 사이트에서는 IBCF에 기반한 알고리즘이 사용되는 것으로 알려져 있다 

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# train, test 데이터 분리
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

### 아이템 기반 CF(Item-Based CF: IBCF)

In [3]:
rating_matrix_t = np.transpose(rating_matrix) # 아이템 간 유사도를 계산해야 하기 때문에 rating_matrix를 전치(transpose)시켜야 한다 
matrix_dummy = rating_matrix_t.copy().fillna(0) # 원래 데이터를 복사하면서 NaN을 0으로 바꾼다

In [4]:
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy) # 아이템 코사인 유사도
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index) # DataFrame으로 변경

In [5]:
# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수, 
# 가중치는 주어진 아이템과 다른 아이템 간의 유사도(item_similarity)
def CF_IBCF(user_id, movie_id):
    if movie_id in item_similarity:      # 현재 영화가 train set에 있는지 확인
        sim_scores = item_similarity[movie_id]         # 현재 영화와 다른 영화의 similarity 값 가져오기       
        user_rating = rating_matrix_t[user_id] # 현 사용자의 모든 rating 값 가져오기
        non_rating_idx = user_rating[user_rating.isnull()].index # 사용자가 평가하지 않은 영화 index 가져오기
        user_rating = user_rating.dropna() # 사용자가 평가하지 않은 영화 제거
        sim_scores = sim_scores.drop(non_rating_idx) # 사용자가 평가하지 않은 영화의 similarity 값 제거
        # 현 영화에 대한 예상 rating 계산, 가중치는 현 영화와 사용자가 평가한 영화의 유사도
        mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

In [6]:
# 정확도 계산
score(CF_IBCF)

1.0125395619113131